In [14]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

In [15]:
# Import necessary libraries
import requests
from bs4 import BeautifulSoup

# Define the URL of the Wikipedia page containing S&P 100 tickers
url = 'https://en.wikipedia.org/wiki/S%26P_100'

# Send an HTTP GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page using BeautifulSoup
    parsed_data = BeautifulSoup(response.content, 'html.parser')

    # Find the table with id 'constituents'
    table = parsed_data.find('table', {'id': 'constituents'})
    
    # Check if the table was found
    if table:
        # Extract S&P 100 tickers from the table
        snp100 = [row.find_all('td')[0].get_text(strip=True) for row in table.find_all('tr') if row.find_all('td')]
    else:
        print("Table not found.")
else:
    # Print an error message if the request was not successful
    print(f"Failed to fetch. Status code: {response.status_code}")

# The variable 'snp100' now contains a list of S&P 100 tickers extracted from the Wikipedia page
snp100

['AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'ADBE',
 'AIG',
 'AMD',
 'AMGN',
 'AMT',
 'AMZN',
 'AVGO',
 'AXP',
 'BA',
 'BAC',
 'BK',
 'BKNG',
 'BLK',
 'BMY',
 'BRK.B',
 'C',
 'CAT',
 'CHTR',
 'CL',
 'CMCSA',
 'COF',
 'COP',
 'COST',
 'CRM',
 'CSCO',
 'CVS',
 'CVX',
 'DE',
 'DHR',
 'DIS',
 'DOW',
 'DUK',
 'EMR',
 'EXC',
 'F',
 'FDX',
 'GD',
 'GE',
 'GILD',
 'GM',
 'GOOG',
 'GOOGL',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'KHC',
 'KO',
 'LIN',
 'LLY',
 'LMT',
 'LOW',
 'MA',
 'MCD',
 'MDLZ',
 'MDT',
 'MET',
 'META',
 'MMM',
 'MO',
 'MRK',
 'MS',
 'MSFT',
 'NEE',
 'NFLX',
 'NKE',
 'NVDA',
 'ORCL',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RTX',
 'SBUX',
 'SCHW',
 'SO',
 'SPG',
 'T',
 'TGT',
 'TMO',
 'TMUS',
 'TSLA',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB',
 'V',
 'VZ',
 'WFC',
 'WMT',
 'XOM']

In [16]:
# Correcting ticker name for Berkshire Hathaway

i = snp100.index('BRK.B')
snp100[i] = 'BRK-B'
snp100

['AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'ADBE',
 'AIG',
 'AMD',
 'AMGN',
 'AMT',
 'AMZN',
 'AVGO',
 'AXP',
 'BA',
 'BAC',
 'BK',
 'BKNG',
 'BLK',
 'BMY',
 'BRK-B',
 'C',
 'CAT',
 'CHTR',
 'CL',
 'CMCSA',
 'COF',
 'COP',
 'COST',
 'CRM',
 'CSCO',
 'CVS',
 'CVX',
 'DE',
 'DHR',
 'DIS',
 'DOW',
 'DUK',
 'EMR',
 'EXC',
 'F',
 'FDX',
 'GD',
 'GE',
 'GILD',
 'GM',
 'GOOG',
 'GOOGL',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'KHC',
 'KO',
 'LIN',
 'LLY',
 'LMT',
 'LOW',
 'MA',
 'MCD',
 'MDLZ',
 'MDT',
 'MET',
 'META',
 'MMM',
 'MO',
 'MRK',
 'MS',
 'MSFT',
 'NEE',
 'NFLX',
 'NKE',
 'NVDA',
 'ORCL',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RTX',
 'SBUX',
 'SCHW',
 'SO',
 'SPG',
 'T',
 'TGT',
 'TMO',
 'TMUS',
 'TSLA',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB',
 'V',
 'VZ',
 'WFC',
 'WMT',
 'XOM']

In [17]:
# Function for preprocessing data of a single ticker to desired format of Zipline

def preprocess(data):
    data.reset_index(inplace=True)
    data.drop(columns=['Adj Close'], inplace=True)
    data.rename(columns={'Date': 'timestamp', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Volume': 'volume', }, inplace=True)
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data['volume'] = data['volume'].astype('float64')
    data.sort_values(by='timestamp', inplace=True)
    data.reset_index(drop=True, inplace=True)

    return data

In [18]:
# Fetching historical data for each ticker in S&P100

start_date = '2010-01-01'
end_date = '2021-07-31'

snp100_data_dict = {}

for ticker in snp100:
    data = yf.download(ticker, start=start_date, end=end_date, interval='1d')
    preprocess(data).to_csv(f'/home/subrina/Documents/SNP100/daily/{ticker}.csv', index=False)
    snp100_data_dict[ticker] = data

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

## Register and Ingest S&P100 Bundle

In [19]:
# from zipline.data import bundles

# bundle_name = 'multi_asset_snp100'

# bundles.register(
#     bundle_name,
#     bundles.csvdir.csvdir_equities(
#         ['daily'],
#         '/home/subrina/Documents/SNP100'
#     ),
#     calendar_name='NYSE',

# )
 
# bundles.bundles

/tmp/ipykernel_6526/1660044892.py:5: UserWarning: Overwriting bundle with name 'multi_asset_snp100'
  bundles.register(


mappingproxy({'quandl': RegisteredBundle(calendar_name='NYSE', start_session=None, end_session=None, minutes_per_day=390, ingest=<function quandl_bundle at 0x7ff6c7041a80>, create_writers=True),
              'quantopian-quandl': RegisteredBundle(calendar_name='NYSE', start_session=None, end_session=None, minutes_per_day=390, ingest=<function quantopian_quandl_bundle at 0x7ff6c7041b20>, create_writers=False),
              'csvdir': RegisteredBundle(calendar_name='NYSE', start_session=None, end_session=None, minutes_per_day=390, ingest=<function csvdir_bundle at 0x7ff6c7041d00>, create_writers=True),
              'multi_asset_snp100': RegisteredBundle(calendar_name='NYSE', start_session=None, end_session=None, minutes_per_day=390, ingest=<bound method CSVDIRBundle.ingest of <zipline.data.bundles.csvdir.CSVDIRBundle object at 0x7ff6c7a1f850>>, create_writers=True)})

In [26]:
bundles.ingest(bundle_name)

In [27]:
bundle_data = bundles.load(bundle_name)
sids = bundle_data.asset_finder.sids
assets = bundle_data.asset_finder.retrieve_all(sids)

print(assets)

[Equity(0 [AAPL]), Equity(1 [ABBV]), Equity(2 [ABT]), Equity(3 [ACN]), Equity(4 [ADBE]), Equity(5 [AIG]), Equity(6 [AMD]), Equity(7 [AMGN]), Equity(8 [AMT]), Equity(9 [AMZN]), Equity(10 [AVGO]), Equity(11 [AXP]), Equity(12 [BA]), Equity(13 [BAC]), Equity(14 [BK]), Equity(15 [BKNG]), Equity(16 [BLK]), Equity(17 [BMY]), Equity(18 [BRK-B]), Equity(19 [C]), Equity(20 [CAT]), Equity(21 [CHTR]), Equity(22 [CL]), Equity(23 [CMCSA]), Equity(24 [COF]), Equity(25 [COP]), Equity(26 [COST]), Equity(27 [CRM]), Equity(28 [CSCO]), Equity(29 [CVS]), Equity(30 [CVX]), Equity(31 [DE]), Equity(32 [DHR]), Equity(33 [DIS]), Equity(34 [DOW]), Equity(35 [DUK]), Equity(36 [EMR]), Equity(37 [EXC]), Equity(38 [F]), Equity(39 [FDX]), Equity(40 [GD]), Equity(41 [GE]), Equity(42 [GILD]), Equity(43 [GM]), Equity(44 [GOOG]), Equity(45 [GOOGL]), Equity(46 [GS]), Equity(47 [HD]), Equity(48 [HON]), Equity(49 [IBM]), Equity(50 [INTC]), Equity(51 [JNJ]), Equity(52 [JPM]), Equity(53 [KHC]), Equity(54 [KO]), Equity(55 [LIN

In [28]:
!zipline bundles

csvdir <no ingestions>
multi_asset_snp100 2023-12-15 07:44:01.699372
multi_asset_snp100 2023-12-15 07:41:38.488699
multi_asset_snp100 2023-12-15 07:32:05.248661
multi_asset_snp100 2023-12-15 07:29:10.693330
multi_asset_snp100 2023-12-15 07:23:30.053949
quandl 2023-12-12 14:51:06.970977
quandl 2023-12-11 18:58:51.384085
quandl 2023-12-11 18:58:18.790981
quandl 2023-12-11 17:27:05.201175
quandl 2023-12-11 17:16:04.140642
quandl 2023-12-11 09:19:57.807125
quandl 2023-12-11 09:18:50.209032
quandl 2023-12-11 09:14:10.865540
quandl 2023-12-11 06:27:30.829838
quandl 2023-12-10 12:31:04.001536
quandl 2023-12-10 12:22:46.640028
quantopian-quandl <no ingestions>


In [25]:
#!zipline ingest -b multi_asset_snp100

[2023-12-15T13:41:39+0600-INFO][zipline.data.bundles.core]
 Ingesting multi_asset_snp100
Loading custom pricing data:   [####################################]  100%
Merging daily equity files:  [####################################]     
[2023-12-15T13:41:40+0600-INFO][root]
 creating /home/subrina/.zipline/data/multi_asset_snp100/2023-12-15T07;41;38.488699/daily_equities.bcolz
[2023-12-15T13:41:40+0600-INFO][root]
 copying /tmp/tmpx73l5huu/multi_asset_snp100/2023-12-15T07;41;38.488699/daily_equities.bcolz/__attrs__ -> /home/subrina/.zipline/data/multi_asset_snp100/2023-12-15T07;41;38.488699/daily_equities.bcolz
[2023-12-15T13:41:40+0600-INFO][root]
 creating /home/subrina/.zipline/data/multi_asset_snp100/2023-12-15T07;41;38.488699/daily_equities.bcolz/close
[2023-12-15T13:41:40+0600-INFO][root]
 copying /tmp/tmpx73l5huu/multi_asset_snp100/2023-12-15T07;41;38.488699/daily_equities.bcolz/close/__attrs__ -> /home/subrina/.zipline/data/multi_asset_snp100/2023-12-15T07;41;38.488699/daily_e